In [2]:
import pandas as pd
data = pd.read_csv("../Data/gpascore.csv")
data.head()

,admit,gre,gpa,rank
0,0,380.0,3.21,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.33,4


#### 칼럼 소개
- admit : 합격여부 (1: 합격, 0: 불합격)
- gre : 영어성적
- gpa: 학점
- rank: 지원한 대학원 수준


### preprocessing (전처리)

In [3]:
data.isnull().sum()

admit    0
gre      1
gpa      0
rank     0
dtype: int64

In [4]:
# 결측치 데이터 삭제
data.dropna(inplace=True)
# 결측치 데이터를 다른값으로 대체
#data.fillna(100)

In [5]:
data.isnull().sum()
# 제거 완료

admit    0
gre      0
gpa      0
rank     0
dtype: int64

In [6]:
# rank 종류 파악
sorted(data['rank'].unique())

[1, 2, 3, 4]

In [8]:
# 영어 점수의 최대값과 최소값
print("최대값:", data['gre'].max())
print("최소값:", data['gre'].min())

최대값: 800.0
최소값: 220.0


In [9]:
# traing data
# 입력층 
x = data[['gre','gpa','rank']]
x.head()

,gre,gpa,rank
0,380.0,3.21,3
1,660.0,3.67,3
2,800.0,4.00,1
3,640.0,3.19,4
4,520.0,2.33,4


In [10]:
# target data
# 합격 여부
# 출력층 
# 0 아니면 1의 이진분류라서 sigmoid
y = data['admit']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: admit, dtype: int64

In [11]:
x.count()

gre     425
gpa     425
rank    425
dtype: int64

> 현재 layer(층)은 하나 => 입력층은 아니고 은닉층과 출력층, 출력층 하나만 있으므로 

---
# Deep Learning Model 만들기

In [12]:
from tensorflow import keras

In [14]:
# keras의 전체 모델에 층을 추가한다.
model = keras.Sequential()
# 1번째 Hidden layer : 256 노드 ( 2의 배수 , 임의의 갯수(hyper parameter))
# activation = sigmoid, tanh, relu, softmax, leakyRelu
# 하나씩 실행하면서 최적의 함수를 찾는다
# input_shape 입력층 
model.add(keras.layers.Dense(256,activation = 'tanh',input_shape=(3,)))
# 출력층 
model.add(keras.layers.Dense(1,activation = 'sigmoid'))

2022-07-19 17:36:41.039036: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
# 손실함수 
model.compile(optimizer="adam",loss='binary_crossentropy',metrics = 'accuracy')
# 모두 경사하강법이나 기울기가 jumping 하는 것이다 
# optimizer : learning rate를 알맞게 조정해줌
# - adam
# - adagrad
# - adadelta
# - rmsprop
# - sgdf : 경사하강법

# loss   => 손실함수
# metrics = 'accuracy'  => 모델 측정기준 : 정확도

#### 이진분류
- last-layer activation : sigmoid
- loss function : binary_crossentropy  => 이진분류 및 확률 예측
#### 다중분류 
- last-layer activation : softmax
- loss function : categorical_crossentropy  => 다중분류 및 확률 예측


<img src="../Data/classification.png" width="90%">

In [20]:
# 학습 시키키
import numpy as np
model.fit(np.array(x),np.array(y), epochs=500)
# verbose = 0 => 진행상황 가리기 

Epoch 1/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5409 - accuracy: 0.7153
Epoch 2/500
14/14 [==============================] - 0s 3ms/step - loss: 0.5560 - accuracy: 0.7059
Epoch 3/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5506 - accuracy: 0.6965
Epoch 4/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5259 - accuracy: 0.7741
Epoch 5/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5247 - accuracy: 0.7741
Epoch 6/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5325 - accuracy: 0.7388
Epoch 7/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5400 - accuracy: 0.7176
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5583 - accuracy: 0.7224
Epoch 9/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5978 - accuracy: 0.6729
Epoch 10/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5583 - accuracy: 0.6941
Epoch 11/

> epoch : 100 => 정확도 0.7365<br>
> epoch : 500 => 정확도 0.7271<br>
> 아직 정제 전 데이터

In [23]:
# 700점 영어성적과 gre와 gpa에 합격률
model.predict(
    [
        [700,3.7,1],
        [700,3.7,2],
        [700,3.7,3],
        [700,3.7,4],
    ]
)


1/1 [==============================] - 0s 39ms/step


array([[0.8616523 ],
       [0.85451835],
       [0.84670955],
       [0.8381672 ]], dtype=float32)

In [22]:
# 400점 영어성적과 gre와 gpa에 합격률 
model.predict(
    [
        [400,2.1,1],
        [400,2.1,2],
        [400,2.1,3],
        [400,2.1,4],
    ]
)


1/1 [==============================] - 0s 37ms/step


array([[0.01672094],
       [0.01518456],
       [0.01381981],
       [0.01260737]], dtype=float32)